In [ ]:
from llm import *

In [2]:
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph
from typing import TypedDict, Annotated
import operator

# 定义状态类型
class GraphState(TypedDict):
    role: str
    query: str
    response: str

# 初始化模型
chat_model = init_chat_model(
    provider="zhipuai",
    model_name="glm-4-plus",
    temperature=0.7
)

# 创建提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个专业的{role}。"),
    ("human", "{query}")
])

# 定义节点函数
def generate_response(state: GraphState) -> GraphState:
    messages = prompt.format_messages(role=state["role"], query=state["query"])
    response = chat_model.invoke(messages)
    state["response"] = response.content
    return state

# 创建图 - 使用 state_schema 参数
workflow = StateGraph(state_schema=GraphState)
workflow.add_node("generate", generate_response)
workflow.set_entry_point("generate")
workflow.set_finish_point("generate")

# 编译图
graph = workflow.compile()

# 运行图
result = graph.invoke({
    "role": "数据科学家",
    "query": "解释一下什么是机器学习？",
    "response": ""  # 初始化为空字符串
})

print(result["response"]) 

当然可以！机器学习是人工智能（AI）的一个子领域，它使计算机系统能够从数据中学习并做出决策或预测，而无需每一步都进行明确的编程。简单来说，机器学习就是让计算机通过经验（数据）来改进其性能的算法和技术。

以下是机器学习的一些关键概念和组成部分：

### 1. **数据**
数据是机器学习的基础。它可以包括各种形式，如数字、文本、图像、声音等。数据的质量和数量直接影响模型的性能。

### 2. **算法**
机器学习算法是用于从数据中学习的规则和模型。常见的算法包括：
- **监督学习**：使用标记的数据（即输入和对应的输出）来训练模型。例如，分类和回归。
- **无监督学习**：使用未标记的数据来发现数据中的模式。例如，聚类和降维。
- **半监督学习**：结合标记和未标记的数据进行学习。
- **强化学习**：通过与环境交互来学习，目标是最大化某种累积奖励。

### 3. **模型**
模型是算法在数据上训练后得到的数学表示。它能够对新数据进行预测或决策。

### 4. **训练**
训练是使用数据来调整模型参数的过程，目的是使模型能够准确地进行预测或分类。

### 5. **评估**
评估是通过测试数据来衡量模型性能的过程。常用的评估指标包括准确率、召回率、F1分数等。

### 6. **过拟合与欠拟合**
- **过拟合**：模型在训练数据上表现很好，但在新数据上表现差，因为它“记住”了训练数据的噪声。
- **欠拟合**：模型在训练数据上表现就很差，因为它没有捕捉到数据的基本结构。

### 7. **特征工程**
特征工程是选择和构造输入变量的过程，这些变量（特征）对模型的性能有重要影响。

### 应用示例
- **推荐系统**：如Netflix的电影推荐。
- **图像识别**：如自动驾驶汽车中的物体识别。
- **自然语言处理**：如Google的翻译服务。

### 工具和库
常用的机器学习工具和库包括TensorFlow、PyTorch、scikit-learn、Keras等。

总之，机器学习通过从数据中自动提取模式和规律，使计算机能够执行复杂的任务，从而在很多领域带来了革命性的变化。


In [3]:
from typing import Annotated, List, TypedDict, Literal
from pydantic import BaseModel, Field
import operator

class SearchQuery(BaseModel):
    search_query: str = Field(None, description="要进行网络搜索的query")
    
class Queries(BaseModel):
    queries: List[SearchQuery] = Field(
        description="搜索query列表",
    )

In [4]:
from duckduckgo_search import AsyncDDGS, DDGS
from functools import lru_cache

@lru_cache()
def web_search(query):
    results = DDGS(proxy=None).text(query, max_results=10)
    return results


def deduplicate_and_format_sources(search_response, max_tokens_per_source):

    sources_list = []
    for response in search_response:
        sources_list.extend(response)
    unique_sources = {source['href']: source for source in sources_list}

    formatted_text = "Sources:\n\n"
    for i, source in enumerate(unique_sources.values(), 1):
        formatted_text += f"Source {source['title']}:\n===\n"
        formatted_text += f"URL: {source['href']}\n===\n"
        formatted_text += f"Most relevant content from source: {source['body']}\n===\n"
    return formatted_text.strip()

In [7]:
report_structure = """使用此结构来创建关于用户提供主题的报告：

1. 引言（无需研究）
   - 主题领域的简要概述

2. 正文部分：
   - 每个部分应关注用户提供主题的一个子主题
   
3. 结论
   - 力求使用1个结构元素（列表或表格）来提炼正文部分的内容
   - 提供报告的简明总结"""


report_planner_query_writer_instructions="""您正在为一份报告进行研究。

<报告主题>
{topic}
</报告主题>

<报告组织结构>
{report_organization}
</报告组织结构>

<任务>
您的目标是生成{number_of_queries}个网络搜索查询，以帮助收集规划报告章节所需的信息。

这些查询应当：

1. 与报告主题相关
2. 有助于满足报告组织结构中指定的要求

请确保查询足够具体，以找到高质量、相关的资源，同时涵盖报告结构所需的广度。
</任务>
"""

report_planner_instructions="""我需要一个简洁且重点突出的报告计划。

<报告主题>
报告的主题是：
{topic}
</报告主题>

<报告组织结构>
报告应遵循以下组织结构：
{report_organization}
</报告组织结构>

<背景信息>
以下是用于规划报告章节的背景信息：
{context}
</背景信息>

<任务>
为报告生成章节列表。您的计划应当紧凑且重点突出，避免章节重叠或不必要的填充内容。

例如，一个良好的报告结构可能如下所示：
1/ 引言
2/ 主题A概述
3/ 主题B概述
4/ A与B的比较
5/ 结论

每个章节应包含以下字段：

- name（名称）- 报告此章节的名称。
- description（描述）- 本章节涵盖的主要主题的简要概述。
- research（研究）- 是否需要为报告的这个章节进行网络研究。
- content（内容）- 章节的内容，现在暂时留空。

整合指南：
- 在主题章节内包含示例和实施细节，而非作为单独章节
- 确保每个章节有明确的目的，内容不重叠
- 合并相关概念而非分开处理

提交前，请检查您的结构，确保没有冗余章节并遵循逻辑流程。
</任务>

<反馈>
以下是审核对报告结构的反馈（如有）：
{feedback}
</反馈>
"""

class Section(BaseModel):
    name: str = Field(
        description="报告此章节的名称。",
    )
    description: str = Field(
        description="本章节将涵盖的主要主题和概念的简要概述。",
    )
    research: bool = Field(
        description="是否需要为报告的这个章节进行网络研究。"
    )
    content: str = Field(
        description="章节的内容。"
    )   

class Sections(BaseModel):
    sections: List[Section] = Field(
        description="报告的章节。",
    )
    
def generate_report_plan():

    topic = "openai deep research"
    
    number_of_queries = 2

    writer_model = init_chat_model(
        provider="zhipuai",
        model_name="glm-4-flash",
        temperature=0.0
    )
    structured_llm = writer_model.with_structured_output(Queries)

    system_instructions_query = report_planner_query_writer_instructions.format(topic=topic, report_organization=report_structure, number_of_queries=number_of_queries)
    results = structured_llm.invoke([SystemMessage(content=system_instructions_query)]+[HumanMessage(content="生成有助于规划报告章节的搜索查询。")])

    query_list = [query.search_query for query in results.queries]
    print(query_list)
    
    search_results = [web_search(query) for query in query_list]
    source_str = deduplicate_and_format_sources(search_results, max_tokens_per_source=1000)

    feedback = ''
    system_instructions_sections = report_planner_instructions.format(topic=topic, report_organization=report_structure, context=source_str, feedback=feedback)


    """
    planner_llm = init_chat_model(
        provider="zhipuai",
        model_name="glm-4-flash",
        temperature=0.0
    )
    
    structured_llm = planner_llm.with_structured_output(Sections)
    report_sections = structured_llm.invoke([SystemMessage(content=system_instructions_sections)]+[HumanMessage(content="生成报告的章节。")])
    sections = report_sections.sections
    """

    planner_llm = init_chat_model(
        provider="zhipuai",
        model_name="glm-4-plus",
        temperature=0.0
    )

    structured_llm = planner_llm.bind_tools([Sections])
    report_sections = structured_llm.invoke([SystemMessage(content=system_instructions_sections)]+[HumanMessage(content="生成报告的章节")])
    tool_call = json.loads(report_sections.content)
    report_sections  = Sections(**tool_call)
    
    sections = report_sections.sections
    return {"sections": sections}

In [8]:
sections = generate_report_plan()
print(sections)

[SystemMessage(content='您正在为一份报告进行研究。\n\n<报告主题>\nopenai deep research\n</报告主题>\n\n<报告组织结构>\n使用此结构来创建关于用户提供主题的报告：\n\n1. 引言（无需研究）\n   - 主题领域的简要概述\n\n2. 正文部分：\n   - 每个部分应关注用户提供主题的一个子主题\n   \n3. 结论\n   - 力求使用1个结构元素（列表或表格）来提炼正文部分的内容\n   - 提供报告的简明总结\n</报告组织结构>\n\n<任务>\n您的目标是生成2个网络搜索查询，以帮助收集规划报告章节所需的信息。\n\n这些查询应当：\n\n1. 与报告主题相关\n2. 有助于满足报告组织结构中指定的要求\n\n请确保查询足够具体，以找到高质量、相关的资源，同时涵盖报告结构所需的广度。\n</任务>\n\n请严格按照以下JSON格式返回结果：\n{\n  "$defs": {\n    "SearchQuery": {\n      "properties": {\n        "search_query": {\n          "default": null,\n          "description": "要进行网络搜索的query",\n          "title": "Search Query",\n          "type": "string"\n        }\n      },\n      "title": "SearchQuery",\n      "type": "object"\n    }\n  },\n  "properties": {\n    "queries": {\n      "description": "搜索query列表",\n      "items": {\n        "$ref": "#/$defs/SearchQuery"\n      },\n      "title": "Queries",\n      "type": "array"\n    }\n  },\n  "required": [\n    "queries"\n  ],\n  "title": "Queries",\n  "type"

In [9]:
query_writer_instructions="""你是一位专业技术写作专家，负责制定有针对性的网络搜索查询，以收集全面的信息用于撰写技术报告章节。

<报告主题>
{topic}
</报告主题>

<章节主题>
{section_topic}
</章节主题>

<任务>
你的目标是生成{number_of_queries}个搜索查询，帮助收集关于上述章节主题的全面信息。

这些查询应当：

1. 与主题相关
2. 探索主题的不同方面
3. 使用与章节主题相同语言

请确保查询足够具体，以找到高质量、相关的信息源。
</任务>
"""


def generate_queries():#(state: SectionState, config: RunnableConfig):
    """ Generate search queries for a report section """

    # topic = state["topic"]
    # section = state["section"]
    topic = 'deep research'
    section = 'deep research使用场景'
    number_of_queries = 3

    writer_model = init_chat_model(
    provider="zhipuai",
    model_name="glm-4-flash",
    temperature=0.0
)
    structured_llm = writer_model.with_structured_output(Queries)

    # Format system instructions
    system_instructions = query_writer_instructions.format(topic=topic, section_topic=section, number_of_queries=number_of_queries)
    queries = structured_llm.invoke([SystemMessage(content=system_instructions)]+[HumanMessage(content="Generate search queries on the provided topic.")])
    print(queries)
    return {"search_queries": queries.queries}

In [10]:
generate_queries()

[SystemMessage(content='你是一位专业技术写作专家，负责制定有针对性的网络搜索查询，以收集全面的信息用于撰写技术报告章节。\n\n<报告主题>\ndeep research\n</报告主题>\n\n<章节主题>\ndeep research使用场景\n</章节主题>\n\n<任务>\n你的目标是生成3个搜索查询，帮助收集关于上述章节主题的全面信息。\n\n这些查询应当：\n\n1. 与主题相关\n2. 探索主题的不同方面\n3. 使用与章节主题相同语言\n\n请确保查询足够具体，以找到高质量、相关的信息源。\n</任务>\n\n请严格按照以下JSON格式返回结果：\n{\n  "$defs": {\n    "SearchQuery": {\n      "properties": {\n        "search_query": {\n          "default": null,\n          "description": "要进行网络搜索的query",\n          "title": "Search Query",\n          "type": "string"\n        }\n      },\n      "title": "SearchQuery",\n      "type": "object"\n    }\n  },\n  "properties": {\n    "queries": {\n      "description": "搜索query列表",\n      "items": {\n        "$ref": "#/$defs/SearchQuery"\n      },\n      "title": "Queries",\n      "type": "array"\n    }\n  },\n  "required": [\n    "queries"\n  ],\n  "title": "Queries",\n  "type": "object"\n}\n\n不要包含任何其他解释，只返回符合上述格式的JSON。\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='Gen

{'search_queries': [SearchQuery(search_query='deep research applications in artificial intelligence'),
  SearchQuery(search_query='deep research use cases in healthcare'),
  SearchQuery(search_query='deep research implementation in financial analysis')]}

In [6]:
from duckduckgo_search import AsyncDDGS, DDGS

def get_result(query):
    results = DDGS(proxy=None).text(query, max_results=10)
    return results


def deduplicate_and_format_sources(search_response, max_tokens_per_source):

    sources_list = []
    for response in search_response:
        sources_list.extend(response)
    unique_sources = {source['href']: source for source in sources_list}

    formatted_text = "Sources:\n\n"
    for i, source in enumerate(unique_sources.values(), 1):
        formatted_text += f"Source {source['title']}:\n===\n"
        formatted_text += f"URL: {source['href']}\n===\n"
        formatted_text += f"Most relevant content from source: {source['body']}\n===\n"
    return formatted_text.strip()

In [ ]:
result = get_result('deep research use cases in healthcare')

In [17]:
def search_web(search_queries):
    
    query_list = [query.search_query for query in search_queries]
    
    search_api = get_config_value(configurable.search_api)

    search_results = [get_result(query) for query in query_list]
    
    source_str = deduplicate_and_format_sources(search_results, max_tokens_per_source=5000, include_raw_content=False)
    return {"source_str": source_str, "search_iterations": state["search_iterations"] + 1}
    
    
def deduplicate_and_format_sources(search_response, max_tokens_per_source):

    sources_list = []
    for response in search_response:
        sources_list.extend(response)
    unique_sources = {source['href']: source for source in sources_list}

    formatted_text = "Sources:\n\n"
    for i, source in enumerate(unique_sources.values(), 1):
        formatted_text += f"Source {source['title']}:\n===\n"
        formatted_text += f"URL: {source['href']}\n===\n"
        formatted_text += f"Most relevant content from source: {source['body']}\n===\n"
    return formatted_text.strip()
    

In [18]:
print(deduplicate_and_format_sources([result],5000))

Sources:

Source 12 Real-Life Applications of Deep Learning in Healthcare in 2025:
===
URL: https://research.aimultiple.com/deep-learning-in-healthcare/
===
Most relevant content from source: The computing capability of deep learning models can enable fast, accurate and efficient operations in patient care, R&D and insurance. Generative AI, computer vision, natural language processing, reinforcement learning are the most commonly used techniques deep learning in healthcare.. IDC claims that:. Research in the pharma industry is one of the fastest growing use cases
===
Source Deep Research and Real World Evidence papers - Use cases and examples ...:
===
URL: https://community.openai.com/t/deep-research-and-real-world-evidence-papers/1112997
===
Most relevant content from source: Interesting discussion below. Do you have any ideas? Deep Research is a powerful new tool designed to autonomously perform multiâ€ step research tasks by gathering, analyzing, and synthesizing information (includ

In [ ]:
section_writer_instructions = """您是一位专业技术作家，正在撰写技术报告的一个章节。

<报告主题>
{topic}
</报告主题>

<章节名称>
{section_name}
</章节名称>

<章节主题>
{section_topic}
</章节主题>

<现有章节内容（如已填写）>
{section_content}
</现有章节内容>

<源材料>
{context}
</源材料>

<写作指南>
1. 如果现有章节内容未填写，请从头开始撰写新章节。
2. 如果现有章节内容已填写，请撰写一个新章节，将现有章节内容与源材料综合起来。
</写作指南>

<长度和风格>
- 严格限制在150-200字
- 不使用营销语言
- 技术重点
- 使用简单、清晰的语言
- 以**粗体**标注您最重要的见解开始
- 使用简短段落（最多2-3句话）
- 使用##作为章节标题（Markdown格式）
- 仅在有助于阐明观点时使用一种结构元素：
  * 要么是比较2-3个关键项目的简明表格（使用Markdown表格语法）
  * 要么是使用正确Markdown列表语法的简短列表（3-5项）：
    - 使用`*`或`-`表示无序列表
    - 使用`1.`表示有序列表
    - 确保正确的缩进和间距
- 以### 参考资料结尾，引用以下源材料，格式为：
  * 列出每个来源的标题、日期和URL
  * 格式：`- 标题：URL`
</长度和风格>

<质量检查>
- 恰好150-200字（不包括标题和来源）
- 谨慎使用仅一种结构元素（表格或列表），且仅在有助于阐明观点时使用
- 一个具体的例子/案例研究
- 以粗体见解开始
- 在创建章节内容前没有前言
- 在末尾引用来源
</质量检查>
"""

section_grader_instructions = """根据指定主题审查报告章节：

<报告主题>
{topic}
</报告主题>

<章节主题>
{section_topic}
</章节主题>

<章节内容>
{section}
</章节内容>

<任务>
评估章节内容是否充分涵盖了章节主题。

如果章节内容未能充分涵盖章节主题，请生成{number_of_follow_up_queries}个后续搜索查询以收集缺失信息。
</任务>

<格式>
    grade: Literal["pass","fail"] = Field(
        description="评估结果，表明响应是否满足要求（'pass'）或需要修改（'fail'）。"
    )
    follow_up_queries: List[SearchQuery] = Field(
        description="后续搜索查询列表。",
    )
</格式>
"""


def write_section(state: SectionState, config: RunnableConfig):
    topic = state["topic"]
    source_str = state["source_str"]

    configurable = Configuration.from_runnable_config(config)

    section = Section(name='Deep Research的功能与特点', description='详细介绍Deep Research的核心功能、技术特点及其与现有AI工具的对比。', research=True, content='')
    
    system_instructions = section_writer_instructions.format(topic=topic, section_title=section.name, section_topic=section.description, context=source_str, section_content=section.content)

    writer_model = init_chat_model(
        provider="zhipuai",
        model_name="glm-4-flash",
        temperature=0.0
    )
    section_content = writer_model.invoke([SystemMessage(content=system_instructions)]+[HumanMessage(content="根据提供的资料生成一个报告章节。")])
    
    # Write content to the section object  
    section.content = section_content.content

    # Grade prompt 
    section_grader_instructions_formatted = section_grader_instructions.format(topic=topic, section_topic=section.description,section=section.content)

    # Feedback 
    structured_llm = writer_model.with_structured_output(Feedback)
    feedback = structured_llm.invoke([SystemMessage(content=section_grader_instructions_formatted)]+[HumanMessage(content="评估报告并考虑针对缺失信息的后续问题：")])

    if feedback.grade == "pass" or state["search_iterations"] >= configurable.max_search_depth:
        # Publish the section to completed sections 
        return  Command(
        update={"completed_sections": [section]},
        goto=END
    )
    else:
        # Update the existing section with new content and update search queries
        return  Command(
        update={"search_queries": feedback.follow_up_queries, "section": section},
        goto="search_web"
        )

In [11]:
sections

{'sections': [Section(name='引言', description='简要概述OpenAI的Deep Research及其在AI领域的重要性。', research=False, content=''),
  Section(name='Deep Research的功能与特点', description='详细介绍Deep Research的核心功能、技术特点及其与现有AI工具的对比。', research=True, content=''),
  Section(name='Deep Research的应用场景', description='探讨Deep Research在不同领域（如金融、科学、政策等）的实际应用案例。', research=True, content=''),
  Section(name='Deep Research的技术架构', description='深入分析Deep Research的技术架构，包括其使用的OpenAI o3模型及其优化方式。', research=True, content=''),
  Section(name='Deep Research的性能评估', description='评估Deep Research的性能，包括其准确性、效率和用户反馈。', research=True, content=''),
  Section(name='Deep Research的未来展望', description='探讨Deep Research未来的发展方向及其对AI领域的潜在影响。', research=True, content=''),
  Section(name='结论', description='总结Deep Research的主要特点和优势，提炼正文部分的关键内容。', research=False, content='')]}

In [ ]:
final_section_writer_instructions="""您是一位专业技术作家，正在撰写一个综合报告其他部分信息的章节。

<报告主题>
{topic}
</报告主题>

<章节名称>
{section_name}
</章节名称>

<章节主题> 
{section_topic}
</章节主题>

<可用报告内容>
{context}
</可用报告内容>

<任务>
1. 章节特定方法：

对于引言：
- 使用#作为报告标题（Markdown格式）
- 限制在50-100字
- 使用简单清晰的语言
- 在1-2段中专注于报告的核心动机
- 使用清晰的叙述结构来介绍报告
- 不包含结构元素（无列表或表格）
- 不需要来源部分

对于结论/总结：
- 使用##作为章节标题（Markdown格式）
- 限制在100-150字
- 对于比较性报告：
    * 必须包含使用Markdown表格语法的重点比较表
    * 表格应提炼报告中的见解
    * 保持表格条目清晰简洁
- 对于非比较性报告： 
    * 仅在有助于提炼报告中的要点时使用一种结构元素：
    * 要么是使用Markdown表格语法比较报告中出现的项目的重点表格
    * 要么是使用正确Markdown列表语法的简短列表：
      - 使用`*`或`-`表示无序列表
      - 使用`1.`表示有序列表
      - 确保正确的缩进和间距
- 以具体的后续步骤或影响结尾
- 不需要来源部分

3. 写作方法：
- 使用具体细节而非一般性陈述
- 让每个词都有价值
- 专注于您最重要的一点
</任务>

<质量检查>
- 对于引言：50-100字限制，#用于报告标题，无结构元素，无来源部分
- 对于结论：100-150字限制，##用于章节标题，最多只有一个结构元素，无来源部分
- Markdown格式
- 不要在回复中包含字数统计或任何前言
</质量检查>"""

In [13]:
result

[{'title': '12 Real-Life Applications of Deep Learning in Healthcare in 2025',
  'href': 'https://research.aimultiple.com/deep-learning-in-healthcare/',
  'body': 'The computing capability of deep learning models can enable fast, accurate and efficient operations in patient care, R&D and insurance. Generative AI, computer vision, natural language processing, reinforcement learning are the most commonly used techniques deep learning in healthcare.. IDC claims that:. Research in the pharma industry is one of the fastest growing use cases'},
 {'title': 'Deep Research and Real World Evidence papers - Use cases and examples ...',
  'href': 'https://community.openai.com/t/deep-research-and-real-world-evidence-papers/1112997',
  'body': 'Interesting discussion below. Do you have any ideas? Deep Research is a powerful new tool designed to autonomously perform multiâ€ step research tasks by gathering, analyzing, and synthesizing information (including from uploaded files) into detailed, cited r